# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

Add support for Selenium scraping for sites that use JavaScript to load content.

In [ ]:
import os
import requests
from scraper import fetch_website_contents, fetch_website_contents_selenium
from IPython.display import Markdown, display
from openai import OpenAI

OLLAMA_BASE_URL = "http://localhost:11434"

In [ ]:
# Check if Ollama is running
requests.get(f'{OLLAMA_BASE_URL}').content

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [ ]:
# Download llama3.2 to your machine 
!ollama pull llama3.2

In [ ]:
# Create OpenAI client with Ollama
ollama = OpenAI(base_url=f'{OLLAMA_BASE_URL}/v1', api_key='ollama')

**Optional (for Selenium scraper):** Run from your terminal: `uv pip install selenium webdriver-manager`

Restart the Jupyter kernel so it picks up the new packages.

That avoids running uv or pip from inside the notebook.

In [ ]:
# Define the WebsiteSummarizer class
class WebsiteSummarizer:
    client = ollama
    system_prompt =  """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""
    user_prompt_prefix =  """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.  
"""

    def __init__(self, url, use_selenium=False):
        self.url = url
        # For websites that use JavaScript to load content, use the Selenium approach
        fetcher = fetch_website_contents_selenium if use_selenium else fetch_website_contents
        self.contents = fetcher(url)
        self.summary = self.summarize()
        return self.display_summary()

    def messages_for(self):
        return [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f'{self.user_prompt_prefix}\n\n{self.contents}'}
        ]

    def summarize(self):
        response = self.client.chat.completions.create(
            model="llama3.2",
            messages=self.messages_for()
        )
        return response.choices[0].message.content
      
    def display_summary(self):
        display(Markdown(self.summary))   

In [ ]:
# Demo
WebsiteSummarizer("https://www.andela.com")

In [ ]:
# Demo - Sites that use JavaScript to load content
WebsiteSummarizer("https://dheerajmaddi.netlify.app/", use_selenium=True)